1. SUMMARY FROM PDF IMAGES USING GROQ
2. COLPALI VECTORSTORE TO INDEX 

In [2]:
from byaldi import RAGMultiModalModel

INDEX_NAME = "finance_data"
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2")

# RAG.index(
#     input_path="finance/", # The path to your documents
#     index_name=INDEX_NAME, # The name you want to give to your index. It'll be saved at `index_root/index_name/`.
#     store_collection_with_index=True, # Whether the index should store the base64 encoded documents.
#     # doc_ids=[0, 1, 2], # Optionally, you can specify a list of document IDs. They must be integers and match the number of documents you're passing. Otherwise, doc_ids will be automatically created.
#     # metadata=[{"author": "John Doe", "date": "2021-01-01"}], # Optionally, you can specify a list of metadata for each document. They must be a list of dictionaries, with the same length as the number of documents you're passing.
#     overwrite=True # Whether to overwrite an index if it already exists. If False, it'll return None and do nothing if `index_root/index_name` exists.
# )

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]


In [4]:
search_index = RAG.from_index(INDEX_NAME)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


You are using in-memory collection. This means every image is stored in memory.
You might want to rethink this if you have a large collection!
Loaded 70 images from 1 JSON files.


/home/recoverx/astarag/VisualFinanceAgent/.venv/lib/python3.10/site-packages/byaldi/colpali.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.ext

In [5]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(), override=True)

In [6]:
res = search_index.search(query="How many companies are leaving Hong Kong",k=1)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


## PDF TO IMAGES

In [12]:
import PyPDF2
import base64
import json
import os

def pdf_to_base64_json_separate(pdf_path, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the PDF file
    for pdf_file in os.listdir(pdf_path):
        curr_pdf_path = os.path.join(pdf_path,pdf_file)
        curr_output_dir = os.path.join(output_dir,pdf_file)
        os.makedirs(curr_output_dir,exist_ok=True)
        with open(curr_pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            num_pages = len(pdf_reader.pages)
            
            # Convert each page to base64 and save to separate JSON files
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                
                # Create a PDF writer object and add the page
                pdf_writer = PyPDF2.PdfWriter()
                pdf_writer.add_page(page)
                
                # Save the single page to a bytes buffer
                from io import BytesIO
                buffer = BytesIO()
                pdf_writer.write(buffer)
                
                # Get the bytes data and encode to base64
                pdf_bytes = buffer.getvalue()
                base64_string = base64.b64encode(pdf_bytes).decode('utf-8')
                
                # Create a dictionary for this page
                page_dict = {f"base_64": base64_string}
                
                # Save to individual JSON file
                output_json_path = os.path.join(curr_output_dir, f"page_{page_num + 1}.json")
                with open(output_json_path, 'w') as json_file:
                    json.dump(page_dict, json_file, indent=2)
                

# Usage
pdf_path = 'finance'
output_dir = 'output_json_files'
pdf_to_base64_json_separate(pdf_path, output_dir)

In [36]:
with open("output_json_files/APAC_Tale of HK SAR _ SNG Report_short version_FINAL.pdf/page_3.json",'r') as file:
    data = json.load(file)

In [37]:
from groq import AsyncGroq
import base64
import os
import asyncio
from typing import List, Optional, Annotated
from pydantic import BaseModel
from io import BytesIO
from PIL import Image



client = AsyncGroq(api_key=os.environ['GROQ_API_KEY'])

def encode_image(base64_string):
    image_bytes = base64.b64decode(base64_string)
    image_stream = BytesIO(image_bytes)

    image = Image.open(image_stream)
    return image

# Data model for LLM to generate
class SummaryResponse(BaseModel):
    summary: Annotated[str, "Summary of the image in 2-3 lines that briefly describes the image"]

# keywords: Annotated[List[str], "List of keywords mentioned in the page"]

async def get_summary(base64_enc: str) -> SummaryResponse:
    completion = await client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Summarize the image in 2-3 lines?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_enc}",
                    },
                },
            ],
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    # response_format={"type": "json_object"},
    stop=None,
    )
    # return SummaryResponse.model_validate_json(chat_completion.choices[0].message.content)
    return completion.choices[0].message.content




recipe = await get_summary(data['base_64'])
# if __name__ == '__main__':
    # summary = asyncio.run(get_summary(data['base_64']))
# print_recipe(recipe)

BadRequestError: Error code: 400 - {'error': {'message': 'invalid image data', 'type': 'invalid_request_error'}}

In [38]:
def convert_base64_to_img(base64_str:str):
    base64_data = base64.b64decode(base64_str)
    image = Image.open(BytesIO(base64_data))
    return image

convert_base64_to_img(data['base_64'])

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f87787fc680>